# Workbook to maintain work ticks

This notebook uses C# interactive shell and SQL API of CosmosDB. It is used to maintain the records of different project execution timer.


## Declaring constants for the notebook



In [11]:
public const string _databaseId = "manual-log";
public const string _containerId = "aws-entries";
public const string _id = "8";
public const string _partitionKey = "EC2";

## Initializing the Cosmos client and container




In [12]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Net;
using Newtonsoft.Json;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

Database dataHost = cosmosClient.GetDatabase(_databaseId);
Container itemHost = dataHost.GetContainer(_containerId);

## Defining the POCO for item



In [14]:
public class LogItem {

    [JsonProperty(PropertyName="id")]
    public string Id {get;set;}

    [JsonProperty(PropertyName="record-name")]
    public string RecordName {get;set;}

    [JsonProperty(PropertyName="tag")]
    public string Tag {get;set;}

    [JsonProperty(PropertyName="start-timestamp")]
    public DateTime? StartedAt {get;set;}

    [JsonProperty(PropertyName="end-timestamp")]
    public DateTime? EndedAt {get;set;}

    [JsonProperty(PropertyName="comment")]
    public string Comment {get;set;}

    [JsonProperty(PropertyName="resource-url")]
    public string ResourceUrl {get;set;}

    [JsonProperty(PropertyName="service")]
    public string ServiceName {get;set;}

    public override string ToString() {
        return $"Log entry is with id- {Id}, name \"{RecordName}\", and started at {StartedAt.ToString()}";
    }
}

## Put data into the structure

This code block should be changed every time you want a new record to be inserted.



In [15]:
LogItem newEntry = new LogItem(){
    Id = _id,
    RecordName = "Tutorial - Hosting a wordpress blog",
    Tag = "Aws-Certification",
    StartedAt = DateTime.UtcNow,
    EndedAt = null,
    Comment = "",
    ResourceUrl = "https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/hosting-wordpress.html",
    ServiceName = _partitionKey
};

## Add the item to container

**CAUTION** skip this code block if you want to only update.


In [9]:
ItemResponse<LogItem> response = await itemHost.CreateItemAsync<LogItem>(newEntry);

Display.AsMarkdown($"Charges of query - **{response.RequestCharge}**");
Display.AsMarkdown($"` {response.Resource.ToString()} `");

Charges of query - **8**

` Log entry is with id- 8, name "Tutorial - Hosting a wordpress blog", and started at 1/11/2021 9:05:27 AM `

## Querying back the item



In [5]:
ItemResponse<LogItem> queryResponse = await itemHost.ReadItemAsync<LogItem>(

    partitionKey: new PartitionKey(_partitionKey),
    id: _id
);

Display.AsMarkdown($"Retreived document: `{queryResponse.Resource}`");
Display.AsMarkdown($"#### Diagnostics for ReadItemAsync:");
Display.AsMarkdown($"` {queryResponse.Diagnostics.ToString()} `");



Retreived document: `Log entry is with id- 7, name "Tutorial - Install LAMP server (Linux 2)", and started at 1/11/2021 8:48:21 AM`

#### Diagnostics for ReadItemAsync:

` {"DiagnosticVersion":"2","Summary":{"StartUtc":"2021-01-11T09:02:09.0130632Z","TotalElapsedTimeInMs":34.8501,"UserAgent":"cosmos-netstandard-sdk/3.12.0|3.11.4|10|X64|Linux 4.19.90-microsoft-standa|.NET Core 3.1.3|","TotalRequestCount":1,"FailedRequestCount":0},"Context":[{"Id":"Microsoft.Azure.Cosmos.Handlers.DiagnosticsHandler","HandlerElapsedTimeInMs":34.4008},{"Id":"SystemInfo","CpuHistory":"(2021-01-11T09:01:32.2975906Z 0.000), (2021-01-11T09:01:42.2938368Z 0.000), (2021-01-11T09:01:42.3074131Z 0.000), (2021-01-11T09:01:52.2959687Z 0.000), (2021-01-11T09:01:52.2960950Z 100.000), (2021-01-11T09:02:02.3029392Z 0.000)"},{"Id":"Microsoft.Azure.Cosmos.Handlers.RetryHandler","HandlerElapsedTimeInMs":34.3967},{"Id":"Microsoft.Azure.Cosmos.Handlers.RouterHandler","HandlerElapsedTimeInMs":34.387},{"Id":"Microsoft.Azure.Cosmos.Handlers.TransportHandler","HandlerElapsedTimeInMs":0.9632},{"Id":"AggregatedClientSideRequestStatistics","ContactedReplicas":[],"RegionsContacted":["https://project-timers-eastus.documents.azure.com:443/"],"FailedReplicas":[]},{"Id":"Microsoft.Azure.Documents.ServerStoreModel","ElapsedTimeInMs":34.2893},{"Id":"StoreResponseStatistics","StartTimeUtc":"2021-01-11T09:02:09.0133477Z","ResponseTimeUtc":"2021-01-11T09:02:09.0475496Z","ElapsedTimeInMs":34.2019,"ResourceType":"Document","OperationType":"Read","LocationEndpoint":"https://project-timers-eastus.documents.azure.com:443/","ActivityId":"d2cafe8a-1bf1-4677-852e-816b8ad1f872","StoreResult":"StorePhysicalAddress: rntbd://cdb-ms-prod-eastus1-fd49.documents.azure.com:14001/apps/8acd1527-a022-4c2a-b2d9-2eea6c3b221d/services/f085975e-f496-48a5-ae43-5ce81ca5fab8/partitions/05064715-9b8e-4a86-8aab-236ca97ea635/replicas/132547803045701811s/, LSN: 14, GlobalCommittedLsn: 13, PartitionKeyRangeId: 0, IsValid: True, StatusCode: 200, SubStatusCode: 0, RequestCharge: 1, ItemLSN: 14, SessionToken: -1#14, UsingLocalLSN: True, TransportException: null"}]} `

## Updating the EndDateTimeStamp for the record


In [7]:
LogItem endItem = (LogItem) queryResponse;

endItem.EndedAt = DateTime.UtcNow;

ItemResponse<LogItem> upsertResponse = await itemHost.UpsertItemAsync(partitionKey: new PartitionKey(_partitionKey), item: endItem);
Display.AsMarkdown($"Charges of query: **{upsertResponse.RequestCharge}**");

Charges of query: **10.67**